<a href="https://colab.research.google.com/github/shashi-sah2003/machine_fault_detection_system/blob/main/machine_fault_detection_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')

In [ ]:
!unzip -q /content/drive/MyDrive/id_00.zip

In [ ]:
!kaggle datasets download -d senaca/mimii-pump-sound-dataset

In [ ]:
!pip install tensorflow tensorflow-io matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import tensorflow as tf

from glob import glob
import librosa
import librosa.display
import IPython.display as ipd
from itertools import cycle

sns.set_theme(style='white',palette=None)
color_pal = plt.rcParams['axes.prop_cycle'].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])


In [ ]:
!unzip -q mimii-pump-sound-dataset.zip

In [ ]:
abnormal_audio_files = glob("/content/id_00/abnormal/*.wav")

normal_audio_files=glob("/content/id_00/normal/*.wav")

In [ ]:
normal_audio_files=normal_audio_files[:500]

print(len(normal_audio_files))
print(len(abnormal_audio_files))

In [ ]:
#Wavelet
import pywt
import numpy as np
import librosa

# Define the wavelet parameters
wavelet = 'db4'  # Choose a specific wavelet (e.g., Daubechies 4)
level = 5  # Level of decomposition

normal_audio_wavelet = []
for i in range(len(normal_audio_files)):
    normal_y_wav, sr = librosa.load(normal_audio_files[i])
    coeffs = pywt.wavedec(normal_y_wav, wavelet, level=level)
    feature = np.concatenate(coeffs)
    normal_audio_wavelet.append(feature)
normal_audio_wavelet = np.array(normal_audio_wavelet)
print(normal_audio_wavelet.shape)
#print(normal_audio_wavelet)

abnormal_audio_wavelet = []
for i in range(len(abnormal_audio_files)):
    abnormal_y_wav, sr = librosa.load(abnormal_audio_files[i])
    coeffs = pywt.wavedec(abnormal_y_wav, wavelet, level=level)
    feature = np.concatenate(coeffs)
    abnormal_audio_wavelet.append(feature)
abnormal_audio_wavelet = np.array(abnormal_audio_wavelet)
print(abnormal_audio_wavelet.shape)
#print(abnormal_audio_wavelet)


In [ ]:
#Kurtosis
import numpy as np
from scipy.stats import kurtosis
normal_audio_wavelet_kurtosis = []
for i in range(normal_audio_wavelet.shape[0]):
    #print(normal_audio_wavelet[i].shape)
    S = librosa.stft(normal_audio_wavelet[i])
    sk = kurtosis(np.abs(S), axis=1)

    # sk = kurtosis(normal_audio_wavelet[i])
    #print(sk)
    normal_audio_wavelet_kurtosis.append(sk)

abnormal_audio_wavelet_kurtosis = []

for i in range(abnormal_audio_wavelet.shape[0]):


    S = librosa.stft(abnormal_audio_wavelet[i])
    sk = kurtosis(np.abs(S), axis=1)

    # sk = kurtosis(normal_audio_wavelet[i])
    #print(sk)
    abnormal_audio_wavelet_kurtosis.append(sk)
print(normal_audio_wavelet)
#print(abnormal_audio_wavelet.shape)




In [ ]:
normal_audio_wavelet_kurtosis= np.array(normal_audio_wavelet_kurtosis)
print(normal_audio_wavelet_kurtosis.shape)
abnormal_audio_wavelet_kurtosis= np.array(abnormal_audio_wavelet_kurtosis)
print(abnormal_audio_wavelet_kurtosis.shape)


In [ ]:
#Spectral centroid
import numpy as np
import librosa

normal_audio_wavelet_centroid = []
for i in range(len(normal_audio_wavelet)):
    S = librosa.stft(normal_audio_wavelet[i])
    centroid = librosa.feature.spectral_centroid(S=np.abs(S))
    normal_audio_wavelet_centroid.append(centroid)
normal_audio_wavelet_centroid = np.array(normal_audio_wavelet_centroid)
print(normal_audio_wavelet_centroid.shape)
print(normal_audio_wavelet_centroid)

abnormal_audio_wavelet_centroid = []
for i in range(len(abnormal_audio_wavelet)):
    S = librosa.stft(abnormal_audio_wavelet[i])
    centroid = librosa.feature.spectral_centroid(S=np.abs(S))
    abnormal_audio_wavelet_centroid.append(centroid)
abnormal_audio_wavelet_centroid = np.array(abnormal_audio_wavelet_centroid)
print(abnormal_audio_wavelet_centroid.shape)
print(abnormal_audio_wavelet_centroid)


In [ ]:
normal_audio_wavelet_centroid = np.reshape(normal_audio_wavelet_centroid, (normal_audio_wavelet_centroid.shape[0], -1))
abnormal_audio_wavelet_centroid = np.reshape(abnormal_audio_wavelet_centroid, (abnormal_audio_wavelet_centroid.shape[0], -1))
print(normal_audio_wavelet_centroid.shape)

(250, 431)


In [ ]:
#ZERO CROSSING RATE

normal_audio_wavelet_zcr = []
for i in range(len(normal_audio_wavelet)):
    # normal_y_zcr, sr = librosa.load(normal_audio_wavelet[i])
    zcr = librosa.feature.zero_crossing_rate(normal_audio_wavelet[i])
    normal_audio_wavelet_zcr.append(zcr)
normal_audio_wavelet_zcr = np.array(normal_audio_wavelet_zcr)
print(normal_audio_wavelet_zcr.shape)
#print(normal_audio_wavelet_zcr)

abnormal_audio_wavelet_zcr = []
for i in range(len(abnormal_audio_wavelet)):
    #y, sr = librosa.load(abnormal_audio_wavelet[i])
    zcr = librosa.feature.zero_crossing_rate(abnormal_audio_wavelet[i])
    abnormal_audio_wavelet_zcr.append(zcr)
abnormal_audio_wavelet_zcr = np.array(abnormal_audio_wavelet_zcr)
print(abnormal_audio_wavelet_zcr.shape)
print(abnormal_audio_wavelet_zcr)

In [ ]:
#MEL Spectrogram
import librosa
import librosa.display
import numpy as np

normal_wav_audio_melspec = []
for audio_data in normal_audio_wavelet:
    melspec = librosa.feature.melspectrogram(y=audio_data, sr=sr)
    normal_wav_audio_melspec.append(melspec)
normal_wav_audio_melspec = np.array(normal_wav_audio_melspec)
#print(abnormal_wav_audio_melspec.shape)
print(normal_wav_audio_melspec)

abnormal_wav_audio_melspec = []
for audio_data in abnormal_audio_wavelet:
    melspec = librosa.feature.melspectrogram(y=audio_data, sr=sr)
    abnormal_wav_audio_melspec.append(melspec)
abnormal_wav_audio_melspec = np.array(abnormal_wav_audio_melspec)
#print(abnormal_wav_audio_melspec.shape)
print(abnormal_wav_audio_melspec)




In [ ]:
normal_wav_audio_melspec = np.reshape(normal_wav_audio_melspec, (normal_wav_audio_melspec.shape[0], -1))
abnormal_wav_audio_melspec = np.reshape(abnormal_wav_audio_melspec, (abnormal_wav_audio_melspec.shape[0], -1))

In [ ]:
print(abnormal_wav_audio_melspec.shape)

(143, 55168)


In [ ]:
normal_contac_all=[]
for i in range(len(normal_audio_wavelet_kurtosis)):
  contac= np.concatenate((normal_audio_wavelet_kurtosis[i],normal_audio_wavelet_zcr[i][0],normal_wav_audio_melspec[i],normal_audio_wavelet_centroid[i]))
  normal_contac_all.append(contac)
normal_contac_all=np.array(normal_contac_all)

abnormal_contac_all=[]
for i in range(len(abnormal_audio_wavelet_kurtosis)):
  ab_contac= np.concatenate((abnormal_audio_wavelet_kurtosis[i],abnormal_audio_wavelet_zcr[i][0],abnormal_wav_audio_melspec[i],abnormal_audio_wavelet_centroid[i]))
  abnormal_contac_all.append(ab_contac)
abnormal_contac_all=np.array(abnormal_contac_all)

In [ ]:
print(normal_contac_all.shape)
print(abnormal_contac_all.shape)

(250, 57055)
(143, 57055)


In [ ]:
y1=np.zeros(normal_contac_all.shape[0])
y2=np.ones(abnormal_contac_all.shape[0])
y_features=np.concatenate((y1, y2))
# print(y_train)

X_features=np.concatenate((normal_contac_all,abnormal_contac_all ))
print(y_features.shape)
print(X_features.shape)

(393,)
(393, 57055)


In [ ]:
# Create a NumPy array
data = X_features
labels=y_features

# Create a NumPy array of labels
# labels = np.array([0, 1, 0, 1])

# Set the random seed for reproducibility
np.random.seed(42)

# Shuffle the indices
indices = np.random.permutation(len(data))

# Shuffle the data and labels based on the shuffled indices
shuffled_data = data[indices]
shuffled_labels = labels[indices]

# Set the desired ratio for train-test split
train_ratio = 0.75

# Calculate the split index
split_index = int(train_ratio * len(data))

# Split the data and labels into train and test sets
train_data_features = shuffled_data[:split_index]
train_labels_features = shuffled_labels[:split_index]
test_data_features = shuffled_data[split_index:]
test_labels_features = shuffled_labels[split_index:]

# Print the train and test sets
print("Training Data:")
print(train_data_features)
print("Training Labels:")
print(train_labels_features)
print("Testing Data:")
print(test_data_features)
print("Testing Labels:")
print(test_labels_features)

In [ ]:
from sklearn.model_selection import train_test_split

# Set the random seed for reproducibility
#np.random.seed(42)

# Split the data and labels into train and test sets
train_data_features, test_data_features, train_labels_features, test_labels_features = train_test_split(
    data, labels, train_size=0.75, random_state=0)

# Print the train and test sets
print("Training Data:")
print(train_data_features)
print("Training Labels:")
print(train_labels_features)
print("Testing Data:")
print(test_data_features)
print("Testing Labels:")
print(test_labels_features)


In [ ]:
#train_data = train_data_features
train_labels = train_labels_features
#test_data = test_data_features
test_labels = test_labels_features

In [ ]:
# Normalize the data
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data_features)
test_data = scaler.transform(test_data_features)

In [ ]:
#Ensemble model of Random forest and XGboost
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

random_forest = RandomForestClassifier()
xgboost = XGBClassifier()

ensemble_model = VotingClassifier(estimators=[('rf', random_forest), ('xgb', xgboost)], voting='hard')
ensemble_model.fit(train_data, train_labels)
ensemble_model.predict(test_data)
ensemble_model.score(test_data, test_labels)


In [ ]:
from sklearn.metrics import f1_score

# Assuming you have already trained and tested the ensemble_model

# Make predictions on the test set
y_pred = ensemble_model.predict(test_data)

# Calculate the F1 score
f1 = f1_score(test_labels, y_pred)

print("F1 Score:", f1)


In [ ]:
#Ensemble model of RFC and SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

random_forest = RandomForestClassifier()
svc = SVC(probability=True)  # Set probability=True for soft voting

ensemble_model = VotingClassifier(estimators=[('rf', random_forest), ('svc', svc)], voting='soft')

ensemble_model.fit(train_data, train_labels)
ensemble_model.predict(test_data)
ensemble_model.score(test_data, test_labels)


In [ ]:
from sklearn.metrics import f1_score

# Assuming you have already trained and tested the ensemble_model

# Make predictions on the test set
y_pred = ensemble_model.predict(test_data_scaled)

# Calculate the F1 score
f1 = f1_score(test_labels, y_pred)

print("F1 Score:", f1)


In [ ]:
#Ensemble model of SVC and XGboost
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

svc = SVC(probability=True)  # Set probability=True for soft voting
xgboost = XGBClassifier()

ensemble_model = VotingClassifier(estimators=[('svc', svc), ('xgb', xgboost)], voting='soft')

ensemble_model.fit(train_data, train_labels)
ensemble_model.predict(test_data)
ensemble_model.score(test_data, test_labels)


In [ ]:
from sklearn.metrics import f1_score

# Assuming you have already trained and tested the ensemble_model

# Make predictions on the test set
y_pred = ensemble_model.predict(test_data)

# Calculate the F1 score
f1 = f1_score(test_labels, y_pred)

print("F1 Score:", f1)

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
# # Generate dummy data
# data = np.random.random((1000, 10))  # Replace with your own data
# labels = np.random.randint(2, size=(1000, 1))  # Replace with your own labels

# Split data into training and testing sets
#train_data = train_data_features
train_labels = train_labels_features
#test_data = test_data_features
test_labels = test_labels_features

# Build the ANN model
model = Sequential()
model.add(Dense(256, input_dim=train_data.shape[1], activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Set the learning rate
# learning_rate = 0.01
# optimizer = Adam(learning_rate=learning_rate)
# Train the model
model.fit(train_data, train_labels, epochs=50, batch_size=16, verbose=1)
# model.fit(train_data, train_labels, verbose=1)
# Evaluate the model
loss, accuracy = model.evaluate(test_data, test_labels, verbose=1)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Assuming you have the predictions for the test set
predictions = model.predict(test_data)

# Calculate the F1 score
f1 = f1_score(test_labels, predictions)

print('F1 Score:', f1)
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix

# Assuming you have the predictions for the test set
predictions = model.predict(test_data)

# Calculate the F1 score
f1 = f1_score(test_labels, predictions)

print('F1 Score:', f1)

# Calculate the confusion matrix
cm = confusion_matrix(test_labels, predictions)
print('Confusion Matrix:')
print(cm)


In [ ]:
# from sklearn.svm import SVC

# # Split data into training and testing sets
# train_data = train_data_features
# train_labels = train_labels_features
# test_data = test_data_features
# test_labels = test_labels_features

# # Build the SVM model
# model = SVC(kernel='rbf', C=1.0, gamma='scale')

# # Train the model
# model.fit(train_data, train_labels)

# # Evaluate the model
# accuracy = model.score(test_data, test_labels)
# print('Test Accuracy:', accuracy)

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# # Normalize the data
# scaler = StandardScaler()
# train_data_scaled = scaler.fit_transform(train_data_features)
# test_data_scaled = scaler.transform(test_data_features)

# Build the SVM model
model = SVC()

# Define the parameter grid for grid search
param_grid = {'C': [0.1, 1, 10],
              'gamma': ['scale', 'auto'],
              'kernel': ['linear', 'rbf']}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(train_data, train_labels_features)

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Perform cross-validation to evaluate the model
cross_val_scores = cross_val_score(best_model, train_data, train_labels_features, cv=5)
mean_cross_val_accuracy = cross_val_scores.mean()

# Train the best model on the entire training data
best_model.fit(train_data, train_labels_features)

# Evaluate the model on the test data
test_accuracy = best_model.score(test_data, test_labels_features)

print('Cross-Validation Accuracy:', mean_cross_val_accuracy)
print('Test Accuracy:', test_accuracy)
print('Best Model:', best_model)



In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Assuming you have the predictions for the test set
predictions = best_model.predict(test_data)

# Calculate the F1 score
f1 = f1_score(test_labels, predictions)

print('F1 Score:', f1)
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix

# Assuming you have the predictions for the test set
predictions = best_model.predict(test_data)

# Calculate the F1 score
f1 = f1_score(test_labels, predictions)

print('F1 Score:', f1)

# Calculate the confusion matrix
cm = confusion_matrix(test_labels, predictions)
print('Confusion Matrix:')
print(cm)


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# # Generate dummy data
# data = np.random.random((1000, 10))  # Replace with your own data
# labels = np.random.randint(2, size=(1000, 1))  # Replace with your own labels

# # Split data into training and testing sets
train_data = train_data_features
train_labels = train_labels_features
test_data = test_data_features
test_labels = test_labels_features

# Reshape data to fit LSTM input shape
train_data = np.reshape(train_data, (train_data.shape[0], 1, train_data.shape[1]))
test_data = np.reshape(test_data, (test_data.shape[0], 1, test_data.shape[1]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(1, train_data.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=25, batch_size=16, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(test_data, test_labels, verbose=1)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Assuming you have the predictions for the test set
predictions = model.predict(test_data)

# Calculate the F1 score
f1 = f1_score(test_labels, predictions)

print('F1 Score:', f1)
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix

# Assuming you have the predictions for the test set
predictions = model.predict(test_data)

# Calculate the F1 score
f1 = f1_score(test_labels, predictions)

print('F1 Score:', f1)

# Calculate the confusion matrix
cm = confusion_matrix(test_labels, predictions)
print('Confusion Matrix:')
print(cm)


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split data into training and testing sets
#train_data = train_data_features
train_labels = train_labels_features
#test_data = test_data_features
test_labels = test_labels_features

# Create a Random Forest classifier
model = RandomForestClassifier(n_estimators=60)

# Train the model
model.fit(train_data, train_labels)

# Make predictions on the test data
predictions = model.predict(test_data)

# Evaluate the model
accuracy = accuracy_score(test_labels, predictions)
print('Test Accuracy:', accuracy)


Test Accuracy: 0.9393939393939394


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

import numpy as np
from sklearn.metrics import f1_score, confusion_matrix

# Assuming you have the predictions for the test set
predictions = model.predict(test_data)

# Calculate the F1 score
f1 = f1_score(test_labels, predictions)

print('F1 Score:', f1)

# Calculate the confusion matrix
cm = confusion_matrix(test_labels, predictions)
print('Confusion Matrix:')
print(cm)



In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Assuming you have the training and testing data and labels
# train_data = train_data_features
train_labels = train_labels_features
# test_data = test_data_features
test_labels = test_labels_features

# Create the XGBoost Classifier model
model = xgb.XGBClassifier()

# Train the model
model.fit(train_data, train_labels)

# Make predictions on the test data
predictions = model.predict(test_data)

# Calculate the accuracy of the model
accuracy = accuracy_score(test_labels, predictions)
print('Accuracy:', accuracy)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Assuming you have the predictions for the test set
predictions = model.predict(test_data)

# Calculate the F1 score
f1 = f1_score(test_labels, predictions)

print('F1 Score:', f1)
# Calculate the confusion matrix
cm = confusion_matrix(test_labels, predictions)
print('Confusion Matrix:')
print(cm)
